The data was obtained from the PubChem database on Human Cannabinoid receptor 1 (CNR1). The dataset consists of 22,886 CNR1 compounds and their bioactivities. The dataset can be downloaded here: https://pubchem.ncbi.nlm.nih.gov/gene/1268#section=Chemicals-and-Bioactivities

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
data_directory = os.path.join("/content/drive/MyDrive/UNTHSC/Github_projects/cannabinoid_project")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pubchempy

In [ ]:
#Load the downloaded dataset
import pandas as pd
import pubchempy as pcp

df = pd.read_csv("/content/drive/MyDrive/UNTHSC/Github_projects/cannabinoid_project/pubchem_geneid_1268_bioactivity_gene.csv")
df.head(5)

,baid,acvalue,aid,sid,cid,geneid,pmid,aidtype,aidmdate,hasdrc,...,cmpdname,targetname,targeturl,ecs,repacxn,taxid,cellids,targettaxid,anatomyid,anatomy
0,101591097,3.500000e-07,328661,103513382,10435654,1268,18363352.0,Confirmatory,20220830,0,...,"Benzo(6,7)cyclohepta(1,2-C)pyrazole-3-carboxam...",CNR1 - cannabinoid receptor 1 (human),/gene/1268,NaN,P21554,9606,NaN,NaN,NaN,NaN
1,101623687,3.500000e-07,404801,103513382,10435654,1268,18512901.0,Confirmatory,20220830,0,...,"Benzo(6,7)cyclohepta(1,2-C)pyrazole-3-carboxam...",CNR1 - cannabinoid receptor 1 (human),/gene/1268,NaN,P21554,9606,NaN,NaN,NaN,NaN
2,363861771,1.000000e-05,1467903,381842384,16661352,1268,29111736.0,Confirmatory,20220830,0,...,CID 16661352,CNR1 - cannabinoid receptor 1 (human),/gene/1268,NaN,P21554,9606,NaN,NaN,NaN,NaN
3,373891693,2.000000e-05,1768789,469802310,164611580,1268,34413936.0,Confirmatory,20220830,0,...,N-(1-bicyclo[2.2.2]octanyl)-3-(4-oxidopyrazin-...,CNR1 - cannabinoid receptor 1 (human),/gene/1268,NaN,P21554,9606,NaN,NaN,NaN,NaN
4,373891698,2.000000e-05,1768789,469821419,164625379,1268,34413936.0,Confirmatory,20220830,0,...,"3-pyrazin-2-yl-N-(3-tricyclo[3.3.1.03,7]nonany...",CNR1 - cannabinoid receptor 1 (human),/gene/1268,NaN,P21554,9606,NaN,NaN,NaN,NaN


In [ ]:
print(df['cid'])

0         10435654
1         10435654
2         16661352
3        164611580
4        164625379
           ...    
22881    134691741
22882    171358032
22883    163183769
22884    171356143
22885    168446849
Name: cid, Length: 22886, dtype: int64


In [ ]:
print(df['acvalue'])

0        3.500000e-07
1        3.500000e-07
2        1.000000e-05
3        2.000000e-05
4        2.000000e-05
             ...     
22881             NaN
22882             NaN
22883             NaN
22884             NaN
22885             NaN
Name: acvalue, Length: 22886, dtype: float64


In [ ]:
#Select first 100 CNR1 compounds from the database
df_new = df.iloc[:100]

In [ ]:
df_new.shape

(100, 27)

In [ ]:
!pip install rdkit-pypi -qqq
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from matplotlib import pyplot as plt
%matplotlib inline
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors


# Webscrape the canonical smiles from PubChemPy

In [ ]:
#CID -- is a PubChem Compound Identification; a non-zero integer PubChem accession identifier for a unique chemical structure
#The cid's in the dataframe was used to collect the SMILES strings of all the compounds from PubChem Database.
#SMILES is a line notation for describing the structure of chemical species

import time
import pubchempy as pcp

# Function to get properties with retry logic and rate limiting
def get_properties_with_retry(cids, retries=5, delay=2):
    for i in range(retries):
        try:
            props = pcp.get_properties('CanonicalSMILES', cids, as_dataframe=True)
            return props
        except pcp.PubChemHTTPError as e:
            print(f"Attempt {i+1} failed: {e}")
            if i < retries - 1:
                time.sleep(delay)
                delay *= 2  # Exponential backoff
            else:
                raise

# List to store SMILES
mol_smiles = []

# Batch size for requests
batch_size = 10

# Iterate over dataframe in batches
for start in range(0, len(df_new), batch_size):
    end = start + batch_size
    batch_cids = df_new['cid'].iloc[start:end].tolist()

    try:
        # Get properties for the current batch
        props = get_properties_with_retry(batch_cids)

        for index, row in props.iterrows():
            mol_smiles.append(row['CanonicalSMILES'])
    except pcp.PubChemHTTPError:
        print(f"Failed to retrieve properties for batch {batch_cids} after several attempts.")

    # Rate limiting: delay between batches
    time.sleep(1)  # Adjust the sleep duration as needed to reduce server load

print(f"Retrieved {len(mol_smiles)} SMILES structures.")


Retrieved 100 SMILES structures.


In [ ]:
#Create a dataframe of the compounds SMILES strings
cann = pd.DataFrame(mol_smiles, columns=['SMILES'])
cann.head(5)

,SMILES
0,C1CCN(CC1)NC(=O)C2=NN(C3=C2CCCC4=C3C=CC(=C4)Cl...
1,C1CCN(CC1)NC(=O)C2=NN(C3=C2CCCC4=C3C=CC(=C4)Cl...
2,CC1=C(N=C(N1C2=CC=C(C=C2)OS(=O)(=O)CCC(F)(F)F)...
3,C1CC2C3=C(CC1O2)C(=NN3C4=NC=C[N+](=C4)[O-])C(=...
4,C1CC2C3=C(CC1O2)C(=NN3C4=NC=CN=C4)C(=O)NC56CC7...


#Visualize the structure of all the selected compounds in a grid from

In [ ]:
!pip install rdkit mols2grid
import mols2grid
mols2grid.display(cann, smiles_col="SMILES")

MolGridWidget()

In [ ]:
#Save the DataFrame of SMILES
cann.to_csv('/content/drive/MyDrive/UNTHSC/Github_projects/cannabinoid_project/cann.csv', index=False)

# Chemical feature extraction using PubChemPy

In [ ]:
#Scrape properties of compounds from PubChem
data = []

for i in cann['SMILES']:
    props = pcp.get_properties(['CanonicalSMILES','MolecularWeight','XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge',
                                'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount',
                                'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount',
                                'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount',
                                'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount',
                                'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D',
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D',
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D',
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D',
                                'EffectiveRotorCount3D', 'ConformerCount3D'], i, 'smiles')
    data.append(props)


In [ ]:
len(data[0][0].keys())

35

In [ ]:
#Append the newly found properties in the dataframe
rows = []
columns = data[0][0].keys()
for i in range(100):
    rows.append(data[i][0].values())
props_df = pd.DataFrame(data=rows, columns=columns)
props_df.head()

,CID,MolecularWeight,CanonicalSMILES,XLogP,ExactMass,MonoisotopicMass,TPSA,Complexity,Charge,HBondDonorCount,...,FeatureCount3D,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D
0,10435654,489.8,C1CCN(CC1)NC(=O)C2=NN(C3=C2CCCC4=C3C=CC(=C4)Cl...,7.0,488.093744,488.093744,50.2,663.0,0.0,1.0,...,8.0,2.0,1.0,0.0,0.0,5.0,0.0,0.8,5.8,10.0
1,10435654,489.8,C1CCN(CC1)NC(=O)C2=NN(C3=C2CCCC4=C3C=CC(=C4)Cl...,7.0,488.093744,488.093744,50.2,663.0,0.0,1.0,...,8.0,2.0,1.0,0.0,0.0,5.0,0.0,0.8,5.8,10.0
2,73301311,620.5,CC1=C(N=C(N1C2=CC=C(C=C2)OS(=O)(=O)CCC(F)(F)F)...,6.3,619.0922320,619.0922320,119.0,964.0,0.0,2.0,...,11.0,4.0,2.0,0.0,1.0,4.0,0.0,1.2,11.2,10.0
3,164611580,395.5,C1CC2C3=C(CC1O2)C(=NN3C4=NC=C[N+](=C4)[O-])C(=...,0.8,395.19573968,395.19573968,94.5,641.0,0.0,1.0,...,10.0,3.0,1.0,1.0,1.0,4.0,0.0,0.8,6.0,10.0
4,164625379,391.5,C1CC2C3=C(CC1O2)C(=NN3C4=NC=CN=C4)C(=O)NC56CC7...,2.0,391.20082506,391.20082506,81.9,689.0,0.0,1.0,...,10.0,4.0,1.0,0.0,1.0,4.0,0.0,0.8,5.8,10.0


In [ ]:
#Add the labels of the selected compounds to the new DataFrame. Activity Value is the label used.
props_df['Activity Value(microM)'] = df['acvalue']

#Save the new DataFrame as csv
props_df.to_csv('/content/drive/MyDrive/UNTHSC/Github_projects/cannabinoid_project/props_df.csv', index=False)
props_df.head()

,CID,MolecularWeight,CanonicalSMILES,XLogP,ExactMass,MonoisotopicMass,TPSA,Complexity,Charge,HBondDonorCount,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,Activity Value(microM)
0,10435654,489.8,C1CCN(CC1)NC(=O)C2=NN(C3=C2CCCC4=C3C=CC(=C4)Cl...,7.0,488.093744,488.093744,50.2,663.0,0.0,1.0,...,2.0,1.0,0.0,0.0,5.0,0.0,0.8,5.8,10.0,3.500000e-07
1,10435654,489.8,C1CCN(CC1)NC(=O)C2=NN(C3=C2CCCC4=C3C=CC(=C4)Cl...,7.0,488.093744,488.093744,50.2,663.0,0.0,1.0,...,2.0,1.0,0.0,0.0,5.0,0.0,0.8,5.8,10.0,3.500000e-07
2,73301311,620.5,CC1=C(N=C(N1C2=CC=C(C=C2)OS(=O)(=O)CCC(F)(F)F)...,6.3,619.0922320,619.0922320,119.0,964.0,0.0,2.0,...,4.0,2.0,0.0,1.0,4.0,0.0,1.2,11.2,10.0,1.000000e-05
3,164611580,395.5,C1CC2C3=C(CC1O2)C(=NN3C4=NC=C[N+](=C4)[O-])C(=...,0.8,395.19573968,395.19573968,94.5,641.0,0.0,1.0,...,3.0,1.0,1.0,1.0,4.0,0.0,0.8,6.0,10.0,2.000000e-05
4,164625379,391.5,C1CC2C3=C(CC1O2)C(=NN3C4=NC=CN=C4)C(=O)NC56CC7...,2.0,391.20082506,391.20082506,81.9,689.0,0.0,1.0,...,4.0,1.0,0.0,1.0,4.0,0.0,0.8,5.8,10.0,2.000000e-05


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm.auto import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
#Removal of rows with NaN values

props_df = props_df.dropna()
props_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CID,86.0,4.728167e+07,4.992739e+07,1.048500e+05,1.153240e+07,1.602696e+07,7.330131e+07,1.646254e+08
XLogP,86.0,5.666279e+00,1.371441e+00,8.000000e-01,4.950000e+00,6.100000e+00,6.500000e+00,7.800000e+00
TPSA,86.0,7.656279e+01,2.528733e+01,3.560000e+01,5.620000e+01,6.980000e+01,1.020000e+02,1.190000e+02
Complexity,86.0,7.294767e+02,1.534040e+02,4.080000e+02,5.850000e+02,6.915000e+02,8.565000e+02,1.060000e+03
Charge,86.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
HBondDonorCount,86.0,1.081395e+00,6.364445e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
HBondAcceptorCount,86.0,5.267442e+00,2.636506e+00,2.000000e+00,3.000000e+00,5.000000e+00,7.000000e+00,1.200000e+01
RotatableBondCount,86.0,5.651163e+00,1.902119e+00,3.000000e+00,4.000000e+00,5.000000e+00,7.750000e+00,1.000000e+01
HeavyAtomCount,86.0,3.359302e+01,4.406887e+00,2.700000e+01,3.000000e+01,3.250000e+01,3.700000e+01,4.300000e+01
IsotopeAtomCount,86.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


# Generate Morgan fingerprints from SMILES

In [ ]:
#Each fingerprint bit corresponds to a fragment of the molecule.This would also be used as features for the predictive model

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors


molecule_list=[]

#Loop through the dataframe and process each row
for index, row in props_df.iterrows():
    smiles=row['CanonicalSMILES']

    #Check if the SMILES notation is valid
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        molecule_list.append(mol)

    else:
        print(f"Invalid SMILES at {index}: {smiles}")
#Generate binary Morgan fingerprints with radius 2

fp =[] #list of fingerprints for molecules
for m in molecule_list:
    fp.append(AllChem.GetMorganFingerprintAsBitVect(m, 2))
print("Fingerprint of the first molecule")
print(list(fp[0]))

Fingerprint of the first molecule
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,

#Convert fingerprints to 2D np.array

In [ ]:
import numpy as np

#Convert fingerprints to 2D np.array
def rdkit_numpy_convert(fp):
    output=[]
    for f in fp:
        arr=np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f,arr)
        output.append(arr)
    return np.asarray(output)

In [ ]:
Finprint = rdkit_numpy_convert(fp) #Matrix of descriptors

In [ ]:
Finprint.shape

(86, 2048)

# Convert Extracted/Original PubChem features to numpy array

In [ ]:
#Drop the Canonical SMILES, Activity Value(microM) and CID
props_new = props_df.drop(columns=['CanonicalSMILES', 'Activity Value(microM)', 'CID'])
props_array= props_new.values
print(props_array)

[['489.8' 7.0 '488.093744' ... 0.8 5.8 10.0]
 ['489.8' 7.0 '488.093744' ... 0.8 5.8 10.0]
 ['620.5' 6.3 '619.0922320' ... 1.2 11.2 10.0]
 ...
 ['463.8' 6.5 '462.078094' ... 0.8 6.2 10.0]
 ['579.5' 6.9 '578.1521321' ... 1.2 11.2 10.0]
 ['537.5' 6.6 '536.1858150' ... 1.2 8.2 10.0]]


#Concatenate the Fingerprint and Molecular descriptors

In [ ]:
combined = np.concatenate((Finprint, props_array), axis=1)
print(combined)

[[0.0 0.0 1.0 ... 0.8 5.8 10.0]
 [0.0 0.0 1.0 ... 0.8 5.8 10.0]
 [0.0 0.0 0.0 ... 1.2 11.2 10.0]
 ...
 [0.0 0.0 1.0 ... 0.8 6.2 10.0]
 [0.0 1.0 1.0 ... 1.2 11.2 10.0]
 [0.0 0.0 0.0 ... 1.2 8.2 10.0]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset

X = combined
y = props_df['Activity Value(microM)'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling of the features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (use the same scaler fitted on the training data)
X_test_scaled = scaler.transform(X_test)

# The scaled data is now ready to be used for model training
print(X_train_scaled[:1])
print(X_test_scaled[:1])

[[-0.12216944  1.87971629 -0.5547002  ... -0.91421265 -0.93740624
   0.17298099]]
[[-0.12216944 -0.53199518 -0.5547002  ...  0.94150258  0.83584246
   0.17298099]]


# Random Forest

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
# Fit the model on the training data
rf_regressor.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# Predict on the test data
y_pred = rf_regressor.predict(X_test_scaled)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 3.0227316732416453e-09


# Support Vector Machine

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize SVR model
svr_model = SVR(kernel='rbf')  # You can choose different kernels like 'linear', 'poly', 'rbf', etc.

# Train the model
svr_model.fit(X_train_scaled, y_train)

# Predict on the testing set
y_pred = svr_model.predict(X_test_scaled)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 5.295633402777715e-09


# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror')  # 'reg:squarederror' for regression

# Train the model
xgb_regressor.fit(X_train_scaled, y_train)

# Predict on the testing set
y_pred = xgb_regressor.predict(X_test_scaled)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 4.336117524483494e-09


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Initialize Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)

# Train the model
dt_regressor.fit(X_train_scaled, y_train)

# Predict on the testing set
y_pred = dt_regressor.predict(X_test_scaled)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 4.480895555555556e-09
